In [ ]:
import numpy as np
import pickle
import TestNN

In [ ]:
output_type = {1:"setosa",2:"versicolor",3:"virginica"}
print (output_type)
print(output_type[1])

output_val = {"setosa" : 1,"versicolor" : 2, "virginica" : 3}
print(output_val["setosa"])

In [ ]:
#Load IRIS Data
with open("iris.csv","r") as f:
    f.readline()
    iris_data = f.readlines()

In [ ]:
X = np.zeros([150,4])
Y = np.zeros([150,3])

In [ ]:
for i in range(len(iris_data)):
    temp = iris_data[i].replace('\n','').split(',')
    X[i] = np.double([temp[0:4]])
    k = output_val[temp[4:5][0]]
    Y[i,k-1] = 1

In [ ]:
print(X[99])
print(Y[99])


In [ ]:
X = np.zeros([150,4])
Y = np.zeros([150,1])

for i in range(len(iris_data)):
    temp = iris_data[i].replace('\n','').split(',')
    X[i] = np.double([temp[0:4]])
    Y[i] = np.double([output_val[temp[4:5][0]]])
    #print(output_val[temp[4:5][0]])

In [ ]:
#print(iris_data[1])
#print(X.shape)
#print(Y.shape)
desc = "This dataset consists of iris data that are classified as 'setosa','versicolor' or 'virginica' with values 1,2 and 3 respectively."
IRIS_DATASET = {"X" : X, "Y" : Y, "DESC" : desc}

In [ ]:
#Save Dataset
with open("IRIS_DATASET.dat","wb") as f:
    pickle.dump(IRIS_DATASET,f)

In [ ]:
#load Dataset
with open("IRIS_DATASET.dat","rb") as f:
    data = pickle.load(f)

In [ ]:
print((data["X"].shape))
print((data["Y"].shape))
print(data["DESC"])
print("\n")
idx = 49
test_data = np.double([data["X"][idx]])
test_data_op = np.double([data["Y"][idx]])

print(test_data)
print(test_data_op)

In [ ]:
#Train IRIS data

net = TestNN.NN([4,5,3])
net.train(data["X"].T, data["Y"].T, 5000, 0.05)

In [ ]:
#Predict IRIS Data

print(net.feedforward(test_data.T))


[[0.10839176]
 [0.18006016]
 [0.45459826]]

In [ ]:
print(op.shape)

In [ ]:
class NN():
    def __init__(self, dim):
        self.layers = len(dim)
        self.dimensions = dim
        self.weight = [np.random.randn(i,j) for i,j in [(x,y) for x,y in zip(dim[1:], dim[:-1])]]
        self.bias = [np.random.randn(y, 1) for y in dim[1:]]
        
    def sigmoid(self,z):
        """The sigmoid function."""
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoid_prime(self,z):
        """Derivative of the sigmoid function."""
        return self.sigmoid(z)*(1-self.sigmoid(z))
    
    def feedforward(self,X):
        A = X
        for w, b in zip(self.weight, self.bias):
            A = self.sigmoid((np.dot(w,A)+b))
        return A
    
    def backprop(self, X,y):
        #print("::::::: Backprop :::::::\n")
        #FeedForward
        nabla_b = [np.zeros(b.shape) for b in self.bias]
        nabla_w = [np.zeros(w.shape) for w in self.weight]
        activation = X
        activations = [X]
        zs = []
        for b, w in zip(self.bias, self.weight):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = self.sigmoid(z)
            activations.append(activation)
            
        # backward pass
        # Delta for Output Layer
        #delta = (activations[-1] - y) * sigmoid_prime(zs[-1]) # For mean squared error                
        delta = (activations[-1] - y) # For Cross Entropy Error (Logistic Error)        
        #nabla_b[-1] = delta
        nabla_b[-1] = (1/X.shape[1]) * np.sum(delta, axis=1, keepdims=True)         
        #nabla_w[-1] =  np.dot(delta, activations[-2].transpose())        
        nabla_w[-1] =  (1/X.shape[1]) * np.dot(delta, activations[-2].transpose())
                
        #Delta for Hidden Layers        
        for l in range(2, self.layers):            
            z = zs[-l]
            sp = self.sigmoid_prime(z)
            delta = np.dot(self.weight[-l+1].transpose(), delta) * sp                       
            nabla_b[-l] = (1/X.shape[1]) * np.sum(delta, axis=1, keepdims=True)
            nabla_w[-l] = (1/X.shape[1]) * np.dot(delta, activations[-l-1].transpose())            
        return (nabla_b, nabla_w)
    
    def update_weights(self,X, y,eta):
        #print("::::::: Update Weights :::::::\n")
        nabla_b = [np.zeros(b.shape) for b in self.bias]
        nabla_w = [np.zeros(w.shape) for w in self.weight]
        
        delta_nabla_b, delta_nabla_w = self.backprop(X, y)              
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        #self.weight = [w-(eta/X.shape[1])*nw for w, nw in zip(self.weight, nabla_w)]   
        #self.bias = [b-(eta/X.shape[1])*nb for b, nb in zip(self.bias, nabla_b)]
        self.weight = [w-(eta/1.0)*nw for w, nw in zip(self.weight, nabla_w)]   
        self.bias = [b-(eta/1.0)*nb for b, nb in zip(self.bias, nabla_b)]
        #print("::::::: End Update Weights :::::::\n")    
        
    def train(self, X, y, epochs=10, eta=0.01):
        for i in range(epochs):
            self.update_weights(X,y,eta)            
        print("Training Complete (Epochs: ",epochs,")")
            
    def predict(self, ip):
        return np.argmax(self.feedforward(ip))
        
    def display(self):
        print("Weights")
        for x in self.weight:
            print(x,x.shape,"\n")
        print("\nBias")
        for y in self.bias:
            print(y, y.shape,"\n")
            
    def display_2(self):
        for b,w in zip(self.bias, self.weight):
            print("Bias: ",b,b.shape,"\n", "Weight: ", w, w.shape)


In [ ]:
temp = [ (i , i**2) for i in range(5)]
print(temp)